In [1]:
import dask.array as da
import dask_image.ndfilters as dif
import dask_ml.cluster

zarr_path = "D:\Sync\Capstone\\10\\volume.zarr"
volume = da.from_zarr("D:\Sync\Capstone\\10\\volume.zarr", "median")

In [19]:
median = dif.median_filter(volume, size=3)
gaussian = dif.gaussian(volume, 2.5)
sobel_x = dif.sobel(gaussian, 2)
sobel_y = dif.sobel(gaussian, 1)
sobel_z = dif.sobel(gaussian, 0)
#sobel_xx = dif.sobel(sobel_x, 2)
#sobel_xy = dif.sobel(sobel_x, 1)
#sobel_xz = dif.sobel(sobel_x, 0)
#sobel_yy = dif.sobel(sobel_y, 1)
#sobel_zz = dif.sobel(sobel_z, 0)
#sobel_yz = dif.sobel(sobel_y, 0)
laplacian = dif.gaussian_laplace(volume, 2.5)
gradient_magnitude = dif.gaussian_gradient_magnitude(volume, 2.5)
labels = da.from_zarr("D:\Sync\Capstone\\10\\volume.zarr", "labels")

In [ ]:
da.to_zarr(gaussian, zarr_path, "gaussian")
da.to_zarr(sobel_x, zarr_path, "sobel_x")
da.to_zarr(sobel_y, zarr_path, "sobel_y")
da.to_zarr(sobel_z, zarr_path, "sobel_z")
#da.to_zarr(sobel_xx, zarr_path, "sobel_xx")
#da.to_zarr(sobel_xy, zarr_path, "sobel_xy")
#da.to_zarr(sobel_xz, zarr_path, "sobel_xz")
#da.to_zarr(sobel_yy, zarr_path, "sobel_yy")
#da.to_zarr(sobel_zz, zarr_path, "sobel_zz")
#da.to_zarr(sobel_yz, zarr_path, "sobel_yz")


In [20]:
arrs = [laplacian, volume]
arrs = [arr.ravel() for arr in arrs]
stacked = da.stack(arrs, -1)
X = stacked.rechunk({1: stacked.shape[1]})

In [21]:
y = labels.ravel()
mask = y != 0

In [22]:
X_train = X[mask].compute()
y_train = y[mask].compute()

In [23]:
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.cluster import MiniBatchKMeans

from dask_ml.wrappers import Incremental
#kmeans = MiniBatchKMeans(n_clusters=2)
sgd = SGDClassifier()
#pa = PassiveAggressiveClassifier()

#kmeans = Incremental(kmeans)
sgd = Incremental(sgd)
#pa = Incremental(pa)

#kmeans.fit(X_train) 
sgd.fit(X_train, y_train, classes=[1,2])
#pa.fit(X_train, y_train, classes=[1,2])

Incremental(estimator=SGDClassifier())

In [24]:
#kmeans_out = kmeans.predict(X).reshape(volume.shape)
sgd_out = sgd.predict(X).reshape(volume.shape) - 1
#pa_out = pa.predict(X).reshape(volume.shape) - 1

In [25]:
from tkinter import filedialog
import napari
import zarr

zarr_file = zarr.open(zarr_path, mode='r+')

viewer = napari.view_image(median, name = "Image")
viewer.add_labels(zarr_file["labels"], name="Labels")
#viewer.add_labels(kmeans_out, name="K-Means")
#viewer.add_labels(1 - kmeans_out, name="K-Means Flipped")
viewer.add_labels(sgd_out, name="SGD")
#viewer.add_labels(pa_out, name="Passive Aggressive")

napari.run()

In [41]:
da.to_zarr(prediction, zarr_path, "pred3")

KeyboardInterrupt: 